In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
log_dir = "tf_logs/run-{}".format(now)

In [4]:
import matplotlib.pyplot as plt

plt.imshow(mnist.train.images[0].reshape(28,28))

In [5]:
mnist.train.labels[0]

7

In [6]:
n_inputs = 784
n_outputs = 5
hidden_size = 100
hidden_num = 5

In [7]:
he_init = tf.variance_scaling_initializer()

def addHiddenLayers(X , hidden_size , hidden_num  , training , rate):
    hidden = list()
    hidden.append(X)
    with tf.name_scope("hiddenLayers"):
        i = 0
        while i < 4*hidden_num:
            hidden.append(tf.layers.dense( hidden[i] , hidden_size , kernel_initializer=he_init , name="hidden{}".format(i+1)))
            hidden.append(tf.layers.batch_normalization(hidden[i+1] , momentum=0.9 , training=training))
            hidden.append(tf.nn.elu(hidden[i+2]))
            hidden.append(tf.layers.dropout(hidden[i+3] , rate , training=training))
            i = i + 4
    return hidden[4*hidden_num ]

In [8]:
X = tf.placeholder(tf.float32 , shape=(None , n_inputs) , name="X")
y = tf.placeholder(tf.int64 , shape=(None) , name="y")

training = tf.placeholder_with_default(True , shape=())

keep_prob = 0.8

with tf.name_scope("DNN"):
    hiddenLast = addHiddenLayers(X=X , hidden_size=hidden_size , hidden_num=hidden_num , training=training , rate=1-keep_prob)
    logits = tf.layers.dense(hiddenLast , n_outputs , activation=None )

with tf.name_scope("Loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y , logits=logits)
    loss = tf.reduce_mean(xentropy)
    
with tf.name_scope("Eval"):
    correct = tf.math.in_top_k(logits , y , 1)
    accuracy = tf.reduce_mean(tf.cast(correct , tf.float64))
    #hiddenLastLayer = tf.

with tf.name_scope("Summary"):
    loss_summary = tf.summary.scalar("loss_sum" , loss)
    acc_summary = tf.summary.scalar("acc_sum" , accuracy)
    file_writer = tf.summary.FileWriter(log_dir , tf.get_default_graph())

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()

saver = tf.train.Saver()

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [9]:
train_zero_four_indicies = mnist.train.labels < 5

ytrain05 = mnist.train.labels[train_zero_four_indicies]
Xtrain05 = mnist.train.images[train_zero_four_indicies]

Xtrain05.shape

(28038, 784)

In [10]:
Xm = Xtrain05.copy()
ym = ytrain05.copy()

def next_batch(step , batch_size):
    global Xm , ym
    start = step * batch_size
    end = start + batch_size
    if end > Xtrain05.shape[0]:
        start = 0
        end = batch_size
        perm = np.random.permutation(Xtrain05.shape[0])
        Xm = Xm[perm]
        ym = ym[perm]
    return Xm[start:end] , ym[start:end]

val_zero_four_indicies = mnist.validation.labels < 5
Xval , yval = mnist.validation.images[val_zero_four_indicies] , mnist.validation.labels[val_zero_four_indicies]

test_zero_four_indicies = mnist.test.labels < 5
Xtest , ytest = mnist.test.images[test_zero_four_indicies] , mnist.test.labels[test_zero_four_indicies]

In [11]:
n_epochs = 100
batch_size = 50
n_batches = Xtrain05.shape[0] // batch_size

In [12]:
with tf.Session() as sess:
    
    sess.run(init)
    #saver.restore(sess , "/tmp/Ch11Ex70.9937731076084841.ckpt")
    acc_val = 0
    acc_pre = 0
    counter = 0
    epoch_with_no_benefits = 0
    for epoch in range(n_epochs):
        acc_pre_epoch = acc_pre
        for step in range(n_batches):
            global_step = step + epoch * n_batches
            Xbatch , ybatch = next_batch(global_step , batch_size)
            updates = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            sess.run([training_op , updates] , feed_dict={X:Xbatch , y:ybatch})
            
            Ls = loss_summary.eval(feed_dict={X:Xbatch , y:ybatch})
            As = acc_summary.eval(feed_dict={X:Xbatch , y:ybatch})
            file_writer.add_summary(Ls , global_step)
            file_writer.add_summary(As , global_step)
            if step % 5 == 0:
                acc_val = sess.run(accuracy , feed_dict={X:Xval , y:yval})
                if acc_val > acc_pre:
                    acc_pre = acc_val
                    saver.save(sess , "/tmp/Ch11Ex7.ckpt")
        if acc_pre == acc_pre_epoch:
            epoch_with_no_benefits = epoch_with_no_benefits + 1
        else :
            epoch_with_no_benefits = 0
        print("Epoch : {} | Validation Accuracy : {} | Best Accuracy Yet : {}".format(epoch + 1, acc_val , acc_pre))
        if epoch_with_no_benefits > 3:
            print("Quitting , {} epoch's trained with no benefits".format(epoch_with_no_benefits))
            break

Epoch : 1 | Validation Accuracy : 0.9667709147771697 | Best Accuracy Yet : 0.9726348709929633


KeyboardInterrupt: 

In [ ]:
with tf.Session() as sess:
    saver.restore(sess , "/tmp/Ch11Ex7.ckpt")
    acc = sess.run(accuracy , feed_dict={X:Xtest , y:ytest , training: False})
    saver.save(sess , "/tmp/Ch11Ex7{}.ckpt".format(acc))
    print("Test Accuracy : {}".format(acc))
    print("Model Saved with new name !")